In [1]:
!pip install pandas
!pip install numpy

/usr/local/opt/python@3.11/bin/python3.11


In [2]:
import warnings
warnings.filterwarnings('ignore')
import os
os.environ["HDF5_USE_FILE_LOCKING"] = "FALSE"
import pandas as pd
import numpy as np

In [8]:
import os
import zipfile

if not os.path.exists("./data/5f9ae242cae5285cd734b91e-momodel"):
    zf = zipfile.ZipFile('./data/5f9ae242cae5285cd734b91e-momodel.zip','r')
    for f in zf.namelist():
        zf.extract(f, './data/')  # 循环解压，将文件解压到指定路径
    zf.close()

data_path = "./data/5f9ae242cae5285cd734b91e-momodel/sms_pub.csv"
sms = pd.read_csv(data_path, encoding='utf-8')
sms.head()


In [161]:
sms.groupby('label').describe()

Unnamed: 0                                                          \
           count           mean            std  min        25%       50%   
label                                                                      
0       636700.0  353916.592548  204369.569092  0.0  176990.75  353887.5   
1        71252.0  354501.890487  204357.472747  1.0  176978.50  354827.5   

                            
             75%       max  
label                       
0      530987.25  707948.0  
1      530786.25  707951.0

In [162]:
def read_stopwords(stopwords_path):
    """
    读取停用词库
    """
    with open(stopwords_path, 'r', encoding='utf-8') as f:
        stopwords = f.read()
    stopwords = stopwords.splitlines()
    return stopwords

In [163]:
stopwords_path = r'./data/5f9ae242cae5285cd734b91e-momodel/scu_stopwords.txt'
# 读取停用词
stopwords = read_stopwords(stopwords_path)
# 展示一些停用词
print(stopwords[-20:])

['嘿', '很', '乎', '会', '或', '既', '及', '啦', '了', '们', '你', '您', '哦', '砰', '啊', '你', '我', '他', '她', '它']


In [164]:
!pip install scikit-learn

In [276]:
from sklearn.model_selection import train_test_split
X = np.array(sms.msg_new)
y = np.array(sms.label)
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42, test_size=0.2)
print("总共的数据大小", X.shape)
print("训练集数据大小", X_train.shape)
print("测试集数据大小", X_test.shape)

总共的数据大小 (707952,)
训练集数据大小 (566361,)
测试集数据大小 (141591,)


In [278]:
from sklearn.feature_extraction.text import CountVectorizer
# 设置匹配的正则表达式和停用词
vect = CountVectorizer(token_pattern=r"(?u)\b\w+\b", stop_words=stopwords)
X_train_dtm = vect.fit_transform(X_train)
X_test_dtm = vect.transform(X_test)

In [279]:
from sklearn.naive_bayes import MultinomialNB
nb = MultinomialNB()
%time nb.fit(X_train_dtm, y_train) 

CPU times: user 153 ms, sys: 29.5 ms, total: 183 ms
Wall time: 181 ms


MultinomialNB()

In [280]:
y_pred = nb.predict(X_test_dtm)
y_pred

array([0, 0, 0, ..., 0, 0, 0])

#### 实验1

##### 文本向量化方式CountVectorizer

In [281]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB

# pipline_list_1用于传给Pipline作为参数
pipeline_list_1 = [
    # --------------------------- 参考代码 ------------------------------
    ('vec', CountVectorizer(token_pattern=r"(?u)\b\w+\b", stop_words=stopwords)),
    ('classifier', MultinomialNB())
    # ------------------------------------------------------------------------
]
# 搭建 pipeline
pipeline = Pipeline(pipeline_list_1)

# 训练 pipeline
pipeline.fit(X_train, y_train)

# 对测试集的数据集进行预测
y_pred = pipeline.predict(X_test)

# 在测试集上进行评估
from sklearn import metrics
print("在测试集上的混淆矩阵：")
print(metrics.confusion_matrix(y_test, y_pred))
print("在测试集上的分类结果报告：")
print(metrics.classification_report(y_test, y_pred))
print("在测试集上的 f1-score ：")
print(metrics.f1_score(y_test, y_pred))

在测试集上的混淆矩阵：
[[125230   2203]
 [   275  13883]]
在测试集上的分类结果报告：
              precision    recall  f1-score   support

           0       1.00      0.98      0.99    127433
           1       0.86      0.98      0.92     14158

    accuracy                           0.98    141591
   macro avg       0.93      0.98      0.95    141591
weighted avg       0.98      0.98      0.98    141591

在测试集上的 f1-score ：
0.918066393334215


In [282]:
import joblib
pipeline_path = 'work/pipeline1.model'
joblib.dump(pipeline, pipeline_path)

['work/pipeline1.model']

In [283]:
# 加载训练好的模型
import joblib
# ------- pipeline 保存的路径，若有变化请修改 --------
pipeline_path = 'work/pipeline1.model'
# --------------------------------------------------
pipeline = joblib.load(pipeline_path)

def predict(message):
    """
    预测短信短信的类别和每个类别的概率
    param: message: 经过jieba分词的短信，如"医生 拿 着 我 的 报告单 说 ： 幸亏 你 来 的 早 啊"
    return: label: 整数类型，短信的类别，0 代表正常，1 代表恶意
            proba: 列表类型，短信属于每个类别的概率，如[0.3, 0.7]，认为短信属于 0 的概率为 0.3，属于 1 的概率为 0.7
    """
    label = pipeline.predict([message])[0]
    proba = list(pipeline.predict_proba([message])[0])
    
    return label, proba

In [284]:
# 数据集的路径
data_private_path = "./data/5f9ae242cae5285cd734b91e-momodel/sms_private.csv"
# 读取数据
sms_private = pd.read_csv(data_private_path, encoding='utf-8')
# 显示前 5 条数据
X_private = np.array(sms_private.msg_new)
print("总共的数据大小", X_private.shape)

总共的数据大小 (78661,)


In [285]:
y_private = pipeline.predict(X_private)

In [286]:
np.save('./work/y_private_pred.npy', y_private)

In [287]:
data = np.load('./work/y_private_pred.npy')

##### 测试数据 结果不正确

In [288]:
predict("代开发票微 v信：dk9330 张经理。可验证后付款，点数优惠！发票保真 ")

(0, [0.6180196559110083, 0.3819803440889922])

#### 测试数据 结果正确

In [289]:
predict("你好，我叫韩美冰，22岁，和几个大学同学兼职赚生活费，在你附近，可上门，可以出示学生证")

(1, [0.12117996116886533, 0.8788200388311346])

#### 实验2

##### 文本向量化方式TfidfVectorizer

In [290]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.preprocessing import MaxAbsScaler

# pipline_list_2用于传给Pipline作为参数
pipeline_list_2 = [
    # --------------------------- 参考代码 ------------------------------
    ('cv',TfidfVectorizer(token_pattern=r"(?u)\b\w+\b", stop_words=stopwords) ),
    ('max_abs_scaler',preprocessing.MaxAbsScaler()),
    ('classifier', MultinomialNB()),
    # ------------------------------------------------------------------------
]

In [291]:

pipeline = Pipeline(pipeline_list_2)

pipeline.fit(X_train, y_train)

# 对测试集的数据集进行预测
y_pred = pipeline.predict(X_test)

# 在测试集上进行评估
from sklearn import metrics
print("在测试集上的混淆矩阵：")
print(metrics.confusion_matrix(y_test, y_pred))
print("在测试集上的分类结果报告：")
print(metrics.classification_report(y_test, y_pred))
print("在测试集上的 f1-score ：")
print(metrics.f1_score(y_test, y_pred))

在测试集上的混淆矩阵：
[[126132   1301]
 [   755  13403]]
在测试集上的分类结果报告：
              precision    recall  f1-score   support

           0       0.99      0.99      0.99    127433
           1       0.91      0.95      0.93     14158

    accuracy                           0.99    141591
   macro avg       0.95      0.97      0.96    141591
weighted avg       0.99      0.99      0.99    141591

在测试集上的 f1-score ：
0.9287644653870141


In [292]:
import joblib
pipeline_path = 'work/pipeline2.model'
joblib.dump(pipeline, pipeline_path)

['work/pipeline2.model']

In [293]:
# 加载训练好的模型
import joblib
# ------- pipeline 保存的路径，若有变化请修改 --------
pipeline_path = 'work/pipeline2.model'
# --------------------------------------------------
pipeline = joblib.load(pipeline_path)

In [294]:
predict("代开发票微 v信：dk9330 张经理。可验证后付款，点数优惠！发票保真")

(1, [0.00010744142063522713, 0.9998925585793683])

In [295]:
predict("你好，我叫韩美冰，22岁，和几个大学同学兼职赚生活费，在你附近，可上门，可以出示学生证")

(1, [0.08394428857734752, 0.9160557114226522])

#### 实验3

##### LR 模型

In [296]:
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
# pipline_list_3用于传给Pipline作为参数
pipeline_list_3 = [
    ('cv',TfidfVectorizer(token_pattern=r"(?u)\b\w+\b", stop_words=stopwords) ),
    ('max_abs_scaler',preprocessing.MaxAbsScaler()),
    ('logistic', LogisticRegression()),
]
# 搭建 pipeline
pipeline = Pipeline(pipeline_list_3)

# 训练 pipeline
pipeline.fit(X_train, y_train)

# 对测试集的数据集进行预测
y_pred = pipeline.predict(X_test)

# 在测试集上进行评估
from sklearn import metrics
print("在测试集上的混淆矩阵：")
print(metrics.confusion_matrix(y_test, y_pred))
print("在测试集上的分类结果报告：")
print(metrics.classification_report(y_test, y_pred))
print("在测试集上的 f1-score ：")
print(metrics.f1_score(y_test, y_pred))

在测试集上的混淆矩阵：
[[127176    257]
 [   684  13474]]
在测试集上的分类结果报告：
              precision    recall  f1-score   support

           0       0.99      1.00      1.00    127433
           1       0.98      0.95      0.97     14158

    accuracy                           0.99    141591
   macro avg       0.99      0.97      0.98    141591
weighted avg       0.99      0.99      0.99    141591

在测试集上的 f1-score ：
0.9662590985693285


In [297]:
import joblib
pipeline_path = 'work/pipeline3.model'
joblib.dump(pipeline, pipeline_path)

['work/pipeline3.model']

In [298]:
# 加载训练好的模型
import joblib
# ------- pipeline 保存的路径，若有变化请修改 --------
pipeline_path = 'work/pipeline3.model'
# --------------------------------------------------
pipeline = joblib.load(pipeline_path)

In [299]:
predict("你好，我叫韩美冰，22岁，和几个大学同学兼职赚生活费，在你附近，可上门，可以出示学生证")

(1, [0.00032297760172217416, 0.9996770223982778])

In [310]:
predict("代开发票微 v信：dk9330 张经理。可验证后付款，点数优惠！发票保真")

(0, [0.9999999973169551, 2.6830446646610556e-09])

#### 实验4

##### BernoulliNB

In [302]:
from sklearn.naive_bayes import BernoulliNB
from sklearn import preprocessing
# pipline_list_2用于传给Pipline作为参数
pipeline_list_4 = [
    ('cv',TfidfVectorizer(token_pattern=r"(?u)\b\w+\b", stop_words=stopwords) ),
    ('max_abs_scaler',preprocessing.MaxAbsScaler()),
    ('classifier', BernoulliNB()),
]
# 搭建 pipeline
pipeline = Pipeline(pipeline_list_4)

# 训练 pipeline
pipeline.fit(X_train, y_train)

# 对测试集的数据集进行预测
y_pred = pipeline.predict(X_test)

# 在测试集上进行评估
from sklearn import metrics
print("在测试集上的混淆矩阵：")
print(metrics.confusion_matrix(y_test, y_pred))
print("在测试集上的分类结果报告：")
print(metrics.classification_report(y_test, y_pred))
print("在测试集上的 f1-score ：")
print(metrics.f1_score(y_test, y_pred))

在测试集上的混淆矩阵：
[[127202    231]
 [   933  13225]]
在测试集上的分类结果报告：
              precision    recall  f1-score   support

           0       0.99      1.00      1.00    127433
           1       0.98      0.93      0.96     14158

    accuracy                           0.99    141591
   macro avg       0.99      0.97      0.98    141591
weighted avg       0.99      0.99      0.99    141591

在测试集上的 f1-score ：
0.9578474686753096


In [303]:
import joblib
pipeline_path = 'work/pipeline4.model'
joblib.dump(pipeline, pipeline_path)
['work/pipeline4.model']

['work/pipeline4.model']

In [307]:
# 加载训练好的模型
import joblib
# ------- pipeline 保存的路径，若有变化请修改 --------
pipeline_path = 'work/pipeline4.model'
# --------------------------------------------------
pipeline = joblib.load(pipeline_path)

In [308]:
predict("你好，我叫韩美冰，22岁，和几个大学同学兼职赚生活费，在你附近，可上门，可以出示学生证")

(0, [0.9999999656485882, 3.435141196185909e-08])

In [309]:
predict("代开发票微 v信：dk9330 张经理。可验证后付款，点数优惠！发票保真")

(0, [0.9999999973169551, 2.6830446646610556e-09])

#### 添加数据

In [139]:
import jieba

strs=["货到付款打消您的顾虑，顺丰包邮 体现我的诚意！   加我微信537392777货不好 生意一次只有一次，好货 您才会常来",
      "你好，我叫韩美冰，22岁，和几个大学同学兼职赚生活费，在你附近，可上门，可以出示学生证",
      "代开发票微 v信：dk9330 张经理。可验证后付款，点数优惠！发票保真 ！增值税普通发票。《工程.建材.广告.会议.咨询.住宿.等各类发票。电话：13661873308张经理 祝：生意兴隆，万事顺意！此广告长期有效敬请保留！如有打扰敬请见谅！短信内不回复请加v"]
for str in strs:
    seg_list = jieba.cut(str,cut_all=True) # 使用paddle模式
    print("Call All Mode: " + ' '.join(list(seg_list)))



Call All Mode: 货到付款 付款 打消 您 的 顾虑 ， 顺 丰 包 邮     体现 我 的 诚意 ！          加 我 微 信 537392777 货 不好     生意 一次 只有 一次 ， 好 货     您 才 会 常来
Call All Mode: 你好 ， 我 叫 韩美 冰 ， 22 岁 ， 和 几个 大学 同学 兼职 赚 生活 生活费 ， 在 你 附近 ， 可上 上门 ， 可以 出示 学生 学生证
Call All Mode: 代 开发 开发票 发票 微     v 信 ： dk9330     张 经理 。 可验证 验证 后 付款 ， 点数 优惠 ！ 发票 保真    ！ 增值 增值税 普通 普通发票 发票 。《 工程 . 建材 . 广告 . 会议 . 咨询 . 住宿 . 等 各类 发票 。 电话 ： 13661873308 张 经理     祝 ： 生意 生意兴隆 意兴 兴隆 ， 万事 顺意 ！ 此 广告 长期 长期有效 有效 敬请 保留 ！ 如有 打扰 敬请 请见 见谅 ！ 短信 内 不 回复 请 加 v
